In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = ['I love my dog', 'I love my cat']

tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)

{'i': 1, 'love': 2, 'my': 3, 'dog': 4, 'cat': 5}
